Requirements

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings


Config

In [ ]:

# Configure settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')


Visualzation

In [ ]:

# Settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12



Loading DataSet

In [ ]:

file_path = 'officesupplies.csv'

try:
    df = pd.read_csv(file_path)
    print("Data loaded successfully!")
    print(f"Dataset shape: {df.shape}")
except FileNotFoundError:
    print(f"File not found at: {file_path}")
   

Looking onto data

In [ ]:
print("First 10 rows of the dataset:")
display(df.head(10))


print("Dataset Information:")
print("=" * 50)
df.info()
print("\nDataset Columns:")
print(df.columns.tolist())

Missing Values

In [ ]:

# Check for missing values
print("Missing Values Analysis:")
print("=" * 50)
missing_data = df.isnull().sum()
missing_percentage = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Values': missing_data,
    'Percentage': missing_percentage
})
display(missing_df[missing_df['Missing Values'] > 0])

if missing_df['Missing Values'].sum() == 0:
    print("No missing values found!")

Looking at Statistics

In [ ]:

# Basic statistics
print("Descriptive Statistics:")
print("=" * 50)
display(df.describe())

# Categorical columns statistics
print("\nCategorical Columns Summary:")
print("=" * 50)
categorical_cols = ['Region', 'Rep', 'Item']
for col in categorical_cols:
    print(f"\n{col}:")
    print(f"Unique values: {df[col].nunique()}")
    print(f"Top 5 values:")
    display(df[col].value_counts().head())

Data Cleaning for better viewings

In [ ]:

# Convert OrderDate to datetime
df['OrderDate'] = pd.to_datetime(df['OrderDate'], errors='coerce')

# Extract date components for analysis
df['Year'] = df['OrderDate'].dt.year
df['Month'] = df['OrderDate'].dt.month
df['Quarter'] = df['OrderDate'].dt.quarter
df['DayOfWeek'] = df['OrderDate'].dt.day_name()
df['MonthName'] = df['OrderDate'].dt.strftime('%B')

Seeing the Revenue of Supply

In [ ]:
df['Revenue'] = df['Units'] * df['Unit Price']

Data after conversion

In [ ]:
print("Date conversion and feature engineering completed!")
print("\nUpdated columns:")
print(df.dtypes)


print("Data after feature engineering:")
display(df.head())


print(f"\nData Quality Check:")
print(f"Rows with invalid dates: {df['OrderDate'].isnull().sum()}")
print(f"Negative units: {(df['Units'] < 0).sum()}")
print(f"Negative prices: {(df['Unit Price'] < 0).sum()}")

More Data Cleaning

In [ ]:

# Remove invalid dates if any
if df['OrderDate'].isnull().sum() > 0:
    df = df.dropna(subset=['OrderDate'])
    print(f"Removed {df['OrderDate'].isnull().sum()} rows with invalid dates")

# Ensure positive values
df = df[(df['Units'] > 0) & (df['Unit Price'] > 0)]
print(f"Removed rows with non-positive units or prices")


EDA

Total Revenue

In [ ]:
total_revenue = df['Revenue'].sum()
total_units = df['Units'].sum()
average_order_value = df['Revenue'].mean()

print("Revenue Overview:")
print("=" * 50)
print(f"Total Revenue: ${total_revenue:,.2f}")
print(f"Total Units Sold: {total_units:,}")
print(f"Average Order Value: ${average_order_value:,.2f}")
print(f"Total Orders: {len(df):,}")
print(f"Average Units per Order: {df['Units'].mean():.2f}")


# Monthly Revenue Trend
plt.figure(figsize=(14, 6))
monthly_revenue = df.groupby(['Year', 'Month'])['Revenue'].sum().reset_index()
monthly_revenue['Date'] = pd.to_datetime(
    monthly_revenue['Year'].astype(str) + '-' + monthly_revenue['Month'].astype(str) + '-01'
)

plt.plot(monthly_revenue['Date'], monthly_revenue['Revenue'], marker='o', linewidth=2)
plt.title('Monthly Revenue Trend', fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('Revenue ($)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


# Quarterly Revenue Analysis
plt.figure(figsize=(10, 6))
quarterly_revenue = df.groupby(['Year', 'Quarter'])['Revenue'].sum().reset_index()
quarterly_revenue['Period'] = quarterly_revenue['Year'].astype(str) + ' Q' + quarterly_revenue['Quarter'].astype(str)

plt.bar(quarterly_revenue['Period'], quarterly_revenue['Revenue'], color='skyblue')
plt.title('Quarterly Revenue Analysis', fontsize=16, fontweight='bold')
plt.xlabel('Quarter', fontsize=12)
plt.ylabel('Revenue ($)', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)


Regional Analysis

In [ ]:

# Regional Performance
plt.figure(figsize=(12, 6))
plt.show()
# Create subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Regional Revenue
regional_revenue = df.groupby('Region')['Revenue'].sum().sort_values(ascending=False)
ax1.bar(regional_revenue.index, regional_revenue.values, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
ax1.set_title('Total Revenue by Region', fontsize=14, fontweight='bold')
ax1.set_xlabel('Region', fontsize=12)
ax1.set_ylabel('Revenue ($)', fontsize=12)
ax1.tick_params(axis='x', rotation=45)
for i, v in enumerate(regional_revenue.values):
    ax1.text(i, v, f'${v:,.0f}', ha='center', va='bottom', fontsize=10)

# Regional Units Sold
regional_units = df.groupby('Region')['Units'].sum().sort_values(ascending=False)
ax2.bar(regional_units.index, regional_units.values, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
ax2.set_title('Total Units Sold by Region', fontsize=14, fontweight='bold')
ax2.set_xlabel('Region', fontsize=12)
ax2.set_ylabel('Units Sold', fontsize=12)
ax2.tick_params(axis='x', rotation=45)
for i, v in enumerate(regional_units.values):
    ax2.text(i, v, f'{v:,.0f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

# Regional Revenue Distribution
plt.figure(figsize=(10, 8))
regional_stats = df.groupby('Region')['Revenue'].agg(['sum', 'mean', 'count']).round(2)
regional_stats.columns = ['Total Revenue', 'Average Order Value', 'Number of Orders']

plt.subplot(2, 2, 1)
plt.pie(regional_stats['Total Revenue'], labels=regional_stats.index, autopct='%1.1f%%', 
        colors=sns.color_palette('Set2'))
plt.title('Revenue Distribution by Region')

plt.subplot(2, 2, 2)
plt.barh(regional_stats.index, regional_stats['Average Order Value'], color='lightcoral')
plt.xlabel('Average Order Value ($)')
plt.title('Average Order Value by Region')
plt.gca().invert_yaxis()

plt.subplot(2, 2, 3)
plt.bar(regional_stats.index, regional_stats['Number of Orders'], color='lightgreen')
plt.xlabel('Region')
plt.ylabel('Number of Orders')
plt.title('Order Count by Region')
plt.xticks(rotation=45)

plt.subplot(2, 2, 4)
regional_avg_price = df.groupby('Region')['Unit Price'].mean()
plt.bar(regional_avg_price.index, regional_avg_price.values, color='lightblue')
plt.xlabel('Region')
plt.ylabel('Average Unit Price ($)')
plt.title('Average Unit Price by Region')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


Product Analysis

In [ ]:


# Top Products by Revenue
plt.figure(figsize=(12, 8))

top_products_revenue = df.groupby('Item')['Revenue'].sum().sort_values(ascending=False).head(10)

plt.subplot(2, 2, 1)
top_products_revenue.plot(kind='barh', color='teal')
plt.xlabel('Revenue ($)')
plt.title('Top 10 Products by Revenue', fontweight='bold')
plt.gca().invert_yaxis()
for i, v in enumerate(top_products_revenue.values):
    plt.text(v, i, f'${v:,.0f}', va='center', fontsize=9)

# Top Products by Units Sold
plt.subplot(2, 2, 2)
top_products_units = df.groupby('Item')['Units'].sum().sort_values(ascending=False).head(10)
top_products_units.plot(kind='barh', color='orange')
plt.xlabel('Units Sold')
plt.title('Top 10 Products by Units Sold', fontweight='bold')
plt.gca().invert_yaxis()
for i, v in enumerate(top_products_units.values):
    plt.text(v, i, f'{v:,.0f}', va='center', fontsize=9)


Price Distribution

In [ ]:

# Price Distribution of Products
plt.subplot(2, 2, 3)
product_prices = df.groupby('Item')['Unit Price'].mean().sort_values(ascending=False).head(10)
product_prices.plot(kind='bar', color='purple')
plt.xlabel('Product')
plt.ylabel('Average Price ($)')
plt.title('Top 10 Most Expensive Products', fontweight='bold')
plt.xticks(rotation=45)


Revenue vs Unit Distribution

In [ ]:

# Revenue vs Units Scatter
plt.subplot(2, 2, 4)
product_summary = df.groupby('Item').agg({
    'Revenue': 'sum',
    'Units': 'sum',
    'Unit Price': 'mean'
}).round(2)

plt.scatter(product_summary['Units'], product_summary['Revenue'], 
            alpha=0.6, s=product_summary['Unit Price']*2, 
            c=product_summary['Unit Price'], cmap='viridis')
plt.xlabel('Total Units Sold')
plt.ylabel('Total Revenue ($)')
plt.title('Product Performance: Units vs Revenue')
plt.colorbar(label='Average Unit Price ($)')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Scatter plot: Units vs Unit Price with Revenue as size
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot([1, 2, 3], [4, 5, 6]) 
plt.show()

Price Distribution Analysis

In [ ]:
# Price Distribution Analysis
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df['Unit Price'], bins=30, edgecolor='black', alpha=0.7, color='royalblue')
plt.xlabel('Unit Price ($)')
plt.ylabel('Frequency')
plt.title('Distribution of Unit Prices')
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
plt.boxplot(df['Unit Price'], vert=False, patch_artist=True)
plt.xlabel('Unit Price ($)')
plt.title('Box Plot of Unit Prices')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

Sales Representative Analysis

In [ ]:
# Sales Rep
plt.figure(figsize=(14, 6))

rep_performance = df.groupby('Rep').agg({
    'Revenue': 'sum',
    'Units': 'sum',
    'OrderDate': 'count'
}).round(2)

rep_performance = rep_performance.rename(columns={'OrderDate': 'OrderCount'})
rep_performance['AverageOrderValue'] = rep_performance['Revenue'] / rep_performance['OrderCount']

# Sort by revenue
rep_performance = rep_performance.sort_values('Revenue', ascending=False)

# Create subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))


# Revenue by Rep
axes[0, 0].barh(range(len(rep_performance)), rep_performance['Revenue'], color='lightblue')
axes[0, 0].set_yticks(range(len(rep_performance)))
axes[0, 0].set_yticklabels(rep_performance.index)
axes[0, 0].set_xlabel('Revenue ($)')
axes[0, 0].set_title('Revenue by Sales Representative', fontweight='bold')
axes[0, 0].invert_yaxis()



# Units Sold by Rep
axes[0, 1].barh(range(len(rep_performance)), rep_performance['Units'], color='lightgreen')
axes[0, 1].set_yticks(range(len(rep_performance)))
axes[0, 1].set_yticklabels([])
axes[0, 1].set_xlabel('Units Sold')
axes[0, 1].set_title('Units Sold by Sales Representative', fontweight='bold')
axes[0, 1].invert_yaxis()



# Order Count by Rep
axes[1, 0].barh(range(len(rep_performance)), rep_performance['OrderCount'], color='lightcoral')
axes[1, 0].set_yticks(range(len(rep_performance)))
axes[1, 0].set_yticklabels(rep_performance.index)
axes[1, 0].set_xlabel('Number of Orders')
axes[1, 0].set_title('Order Count by Sales Representative', fontweight='bold')
axes[1, 0].invert_yaxis()


# Average Order Value by Rep
axes[1, 1].barh(range(len(rep_performance)), rep_performance['AverageOrderValue'], color='gold')
axes[1, 1].set_yticks(range(len(rep_performance)))
axes[1, 1].set_yticklabels([])
axes[1, 1].set_xlabel('Average Order Value ($)')
axes[1, 1].set_title('Average Order Value by Sales Representative', fontweight='bold')
axes[1, 1].invert_yaxis()

plt.tight_layout()
plt.show()




Regional graphs

In [ ]:

# Regional Distribution of Sales Reps
plt.figure(figsize=(12, 6))

# Create a pivot table for sales reps by region
rep_region_pivot = pd.crosstab(df['Rep'], df['Region'], values=df['Revenue'], aggfunc='sum')

plt.imshow(rep_region_pivot, cmap='YlOrRd', aspect='auto')
plt.colorbar(label='Revenue ($)')
plt.xlabel('Region')
plt.ylabel('Sales Representative')
plt.title('Sales Representative Performance by Region', fontsize=14, fontweight='bold')
plt.xticks(range(len(rep_region_pivot.columns)), rep_region_pivot.columns)
plt.yticks(range(len(rep_region_pivot.index)), rep_region_pivot.index)

# Add text annotations
for i in range(len(rep_region_pivot.index)):
    for j in range(len(rep_region_pivot.columns)):
        revenue = rep_region_pivot.iloc[i, j]
        if not pd.isna(revenue) and revenue > 0:
            plt.text(j, i, f'${revenue:,.0f}', ha='center', va='center', 
                    fontsize=8, color='black' if revenue < rep_region_pivot.values.max()/2 else 'white')

plt.tight_layout()
plt.show()

Time-Based

In [ ]:

# Day of Week Analysis
plt.figure(figsize=(14, 5))

# Define day order
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Convert to categorical with correct order
df['DayOfWeek'] = pd.Categorical(df['DayOfWeek'], categories=day_order, ordered=True)

# Group by day of week
day_stats = df.groupby('DayOfWeek').agg({
    'Revenue': 'sum',
    'Units': 'sum',
    'OrderDate': 'count'
}).reindex(day_order)

# Create subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Revenue by Day
axes[0].bar(day_stats.index, day_stats['Revenue'], color='skyblue')
axes[0].set_title('Revenue by Day of Week', fontweight='bold')
axes[0].set_xlabel('Day')
axes[0].set_ylabel('Revenue ($)')
axes[0].tick_params(axis='x', rotation=45)
for i, v in enumerate(day_stats['Revenue']):
    axes[0].text(i, v, f'${v:,.0f}', ha='center', va='bottom', fontsize=9)

# Units Sold by Day
axes[1].bar(day_stats.index, day_stats['Units'], color='lightgreen')
axes[1].set_title('Units Sold by Day of Week', fontweight='bold')
axes[1].set_xlabel('Day')
axes[1].set_ylabel('Units Sold')
axes[1].tick_params(axis='x', rotation=45)
for i, v in enumerate(day_stats['Units']):
    axes[1].text(i, v, f'{v:,.0f}', ha='center', va='bottom', fontsize=9)

# Order Count by Day
axes[2].bar(day_stats.index, day_stats['OrderDate'], color='salmon')
axes[2].set_title('Order Count by Day of Week', fontweight='bold')
axes[2].set_xlabel('Day')
axes[2].set_ylabel('Number of Orders')
axes[2].tick_params(axis='x', rotation=45)
for i, v in enumerate(day_stats['OrderDate']):
    axes[2].text(i, v, f'{v:,.0f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

Monthly

In [ ]:

# Monthly Patterns
plt.figure(figsize=(14, 6))

monthly_patterns = df.groupby(['MonthName', 'Month']).agg({
    'Revenue': 'sum',
    'Units': 'sum'
}).reset_index().sort_values('Month')

# Define month order
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']
monthly_patterns['MonthName'] = pd.Categorical(monthly_patterns['MonthName'], 
                                                categories=month_order, ordered=True)
monthly_patterns = monthly_patterns.sort_values('MonthName')

# Create subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Revenue by Month
ax1.bar(monthly_patterns['MonthName'], monthly_patterns['Revenue'], color='royalblue')
ax1.set_title('Monthly Revenue Pattern', fontsize=14, fontweight='bold')
ax1.set_ylabel('Revenue ($)', fontsize=12)
ax1.tick_params(axis='x', rotation=45)
ax1.grid(axis='y', alpha=0.3)
for i, v in enumerate(monthly_patterns['Revenue']):
    ax1.text(i, v, f'${v:,.0f}', ha='center', va='bottom', fontsize=9)

# Units Sold by Month
ax2.bar(monthly_patterns['MonthName'], monthly_patterns['Units'], color='darkorange')
ax2.set_title('Monthly Units Sold Pattern', fontsize=14, fontweight='bold')
ax2.set_ylabel('Units Sold', fontsize=12)
ax2.set_xlabel('Month', fontsize=12)
ax2.tick_params(axis='x', rotation=45)
ax2.grid(axis='y', alpha=0.3)
for i, v in enumerate(monthly_patterns['Units']):
    ax2.text(i, v, f'{v:,.0f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()



Correlation Analysis and HeatMap

In [ ]:
# Correlation Matrix
plt.figure(figsize=(10, 8))

# Select numerical columns for correlation
numerical_cols = ['Units', 'Unit Price', 'Revenue', 'Year', 'Month', 'Quarter']
correlation_data = df[numerical_cols]

# Calculate correlation matrix
corr_matrix = correlation_data.corr()

# Heatmap
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix of Numerical Variables', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()




Analytics & Insights


In [ ]:
# Calculate key performance metrics
print(" KEY PERFORMANCE INDICATORS (KPIs)")


# Revenue metrics
monthly_revenue_growth = monthly_revenue['Revenue'].pct_change().mean() * 100
best_month = monthly_revenue.loc[monthly_revenue['Revenue'].idxmax()]
worst_month = monthly_revenue.loc[monthly_revenue['Revenue'].idxmin()]

print(f" Revenue Metrics:")
print(f"  • Monthly Revenue Growth Rate: {monthly_revenue_growth:.2f}%")
print(f"  • Best Month: {best_month['Date'].strftime('%B %Y')} - ${best_month['Revenue']:,.2f}")
print(f"  • Worst Month: {worst_month['Date'].strftime('%B %Y')} - ${worst_month['Revenue']:,.2f}")


Regional metrics

In [ ]:

# Regional metrics
best_region = regional_revenue.idxmax()
worst_region = regional_revenue.idxmin()
region_revenue_ratio = regional_revenue.max() / regional_revenue.min()

print(f"\n Regional Metrics:")
print(f"  • Best Performing Region: {best_region} - ${regional_revenue[best_region]:,.2f}")
print(f"  • Worst Performing Region: {worst_region} - ${regional_revenue[worst_region]:,.2f}")
print(f"  • Best:Worst Revenue Ratio: {region_revenue_ratio:.2f}")


Product metrics

In [ ]:

# Product metrics
best_product_revenue = top_products_revenue.idxmax()
best_product_units = top_products_units.idxmax()
avg_price = df['Unit Price'].mean()
price_std = df['Unit Price'].std()

print(f"\n Product Metrics:")
print(f"  • Best Product by Revenue: {best_product_revenue}")
print(f"  • Best Product by Units: {best_product_units}")
print(f"  • Average Unit Price: ${avg_price:.2f}")
print(f"  • Price Standard Deviation: ${price_std:.2f}")


Sales rep metrics

In [ ]:


best_rep = rep_performance['Revenue'].idxmax()
worst_rep = rep_performance['Revenue'].idxmin()
rep_performance_ratio = rep_performance['Revenue'].max() / rep_performance['Revenue'].min()

print(f"\nSales Representative Metrics:")
print(f"  • Top Performer: {best_rep} - ${rep_performance.loc[best_rep, 'Revenue']:,.2f}")
print(f"  • Lowest Performer: {worst_rep} - ${rep_performance.loc[worst_rep, 'Revenue']:,.2f}")
print(f"  • Performance Ratio (Best:Worst): {rep_performance_ratio:.2f}")



Pareto Analysis

In [ ]:

# Identify top 20% products that generate 80% revenue
print("\n PARETO ANALYSIS (80/20 Rule)")


product_revenue_sorted = df.groupby('Item')['Revenue'].sum().sort_values(ascending=False)
product_revenue_sorted_cumsum = product_revenue_sorted.cumsum()
total_revenue = product_revenue_sorted.sum()
pareto_threshold = 0.8 * total_revenue


 Find number of products that reach 80% revenue

In [ ]:


pareto_products = product_revenue_sorted_cumsum[product_revenue_sorted_cumsum <= pareto_threshold]
top_n_products = len(pareto_products)
total_products = len(product_revenue_sorted)

print(f"Total Products: {total_products}")
print(f"Products generating 80% revenue: {top_n_products}")
print(f"Percentage of products: {(top_n_products/total_products)*100:.1f}%")
print(f"\nTop {top_n_products} Products generating 80% revenue:")
for i, (product, revenue) in enumerate(product_revenue_sorted.head(top_n_products).items()):
    print(f"  {i+1}. {product}: ${revenue:,.2f} ({(revenue/total_revenue)*100:.1f}%)")




 Seasonality Analysis

In [ ]:

print("\n SEASONALITY ANALYSIS")


# Calculate seasonal indices
seasonal_index = monthly_patterns.copy()
seasonal_index['Revenue_Index'] = (seasonal_index['Revenue'] / seasonal_index['Revenue'].mean()) * 100
seasonal_index['Units_Index'] = (seasonal_index['Units'] / seasonal_index['Units'].mean()) * 100

print("Monthly Seasonal Indices (100 = Average):")
display(seasonal_index[['MonthName', 'Revenue', 'Revenue_Index', 'Units', 'Units_Index']])


 Identify peak and off-peak seasons

In [ ]:


peak_season = seasonal_index.loc[seasonal_index['Revenue_Index'].idxmax()]
off_peak_season = seasonal_index.loc[seasonal_index['Revenue_Index'].idxmin()]

print(f"\nPeak Season: {peak_season['MonthName']} (Index: {peak_season['Revenue_Index']:.1f})")
print(f"Off-Peak Season: {off_peak_season['MonthName']} (Index: {off_peak_season['Revenue_Index']:.1f})")
summary_stats = pd.DataFrame({
    'Metric': [
        'Total Revenue',
        'Total Units Sold',
        'Total Orders',
        'Average Order Value',
        'Average Units per Order',
        'Average Unit Price',
        'Number of Regions',
        'Number of Products',
        'Number of Sales Reps',
        'Date Range Start',
        'Date Range End',
        'Monthly Revenue Growth Rate'
    ],
    'Value': [
        f"${total_revenue:,.2f}",
        f"{total_units:,}",
        f"{len(df):,}",
        f"${average_order_value:,.2f}",
        f"{df['Units'].mean():.2f}",
        f"${df['Unit Price'].mean():.2f}",
        f"{df['Region'].nunique()}",
        f"{df['Item'].nunique()}",
        f"{df['Rep'].nunique()}",
        f"{df['OrderDate'].min().strftime('%Y-%m-%d')}",
        f"{df['OrderDate'].max().strftime('%Y-%m-%d')}",
        f"{monthly_revenue_growth:.2f}%"
    ]
})

print(" EXECUTIVE SUMMARY")

display(summary_stats)



final insights report

In [ ]:



print("KEY BUSINESS INSIGHTS & RECOMMENDATIONS")


print("\n1.  REVENUE TRENDS:")
print(f"   • Revenue shows {'positive' if monthly_revenue_growth > 0 else 'negative'} growth trend")
print(f"   • Best performing quarter: Q{quarterly_revenue.loc[quarterly_revenue['Revenue'].idxmax(), 'Quarter']}")
print(f"   • Peak revenue month: {best_month['Date'].strftime('%B')}")

print("\n2.  REGIONAL PERFORMANCE:")
print(f"   • {best_region} region is the top performer")
print(f"   • Consider allocating more resources to {best_region}")
print(f"   • {worst_region} region needs performance improvement strategies")

print("\n3.  PRODUCT STRATEGY:")
print(f"   • Focus on promoting {best_product_revenue} (highest revenue generator)")
print(f"   • {best_product_units} has the highest sales volume")
print(f"   • Only {top_n_products} out of {total_products} products generate 80% of revenue")

print("\n4.  SALES TEAM OPTIMIZATION:")
print(f"   • {best_rep} is the top-performing sales representative")
print(f"   • Performance variance between reps: {rep_performance_ratio:.1f}x")
print(f"   • Consider cross-training and sharing best practices")

print("\n5.  TIMING & SEASONALITY:")
print(f"   • Best day for sales: {day_stats['Revenue'].idxmax()}")
print(f"   • Peak season: {peak_season['MonthName']}")
print(f"   • Plan inventory and promotions around seasonal patterns")

print("\n6.  PRICING INSIGHTS:")
print(f"   • Average unit price: ${avg_price:.2f}")
print(f"   • Consider dynamic pricing strategies")
print(f"   • Monitor price elasticity for optimal pricing")


print(" RECOMMENDED ACTIONS:")
                           
print("1. Increase marketing focus during peak seasons")
print("2. Implement regional-specific strategies")
print("3. Optimize product mix based on Pareto analysis")
print("4. Develop sales training programs based on top performers' methods")
print("5. Consider promotional campaigns for underperforming regions/products")
print("6. Implement dynamic pricing based on demand patterns")




In [ ]:


print(" ANALYSIS COMPLETE!")

print(f"• Data analyzed: {len(df):,} records")
     